# INSTALL LIBRARIES 

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip3 install twint
!pip install -U textblob
!pip install emoji --upgrade
!pip install --upgrade pip
!pip install wordcloud
!pip install nest_asyncio

In [ ]:
#ADD THIS INSTALL
!pip3 install --user --upgrade git+https://github.com/yunusemrecatalcam/twint.git@twitter_legacy2

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
import twint

# SCRAP DATA

In [ ]:
#configuration
config = twint.Config()
config.Search = "Walmart Canada price matching"
config.Lang = "en"
config.Since = "2020-07-01"
config.Until = "2020-10-08"
config.Limit = 1000
config.Format = 'Tweet id: {id} | Date: {date} | Time: {time} | Tweet: {tweet}'
config.Store_csv = True
# config.Output crash the code, not good anymore
# config.Output = "Travel_Tourism.csv"


In [ ]:
twint.run.Search(config)
# Add the below the store scrapping data in list
tlist1 = config.search_tweet_list
# Print out when done the process of scrapping
print("Scrapping is completed")

In [ ]:
# Convert the list to dataframe
PriceMatching_df = pd.DataFrame(tlist1)

In [ ]:
# Get datafram info
PriceMatching_df.info()

In [ ]:
# Save to excel file
PriceMatching_df.to_excel("PriceMatching.xlsx")

In [ ]:
#configuration
config = twint.Config()
config.Search = "COVID-19 Walmart Canada price matching"
config.Lang = "en"
config.Since = "2020-07-01"
config.Until = "2020-09-28"
config.Limit = 1000
config.Format = 'Tweet id: {id} | Date: {date} | Time: {time} | Tweet: {tweet}'
config.Store_csv = True

In [ ]:
twint.run.Search(config)
# Add the below the store scrapping data in list
tlist2 = config.search_tweet_list
# Print out when done the process of scrapping
print("Scrapping is completed")

In [ ]:
# Convert the list to dataframe
CoronavirusWalmartPriceMatching_df = pd.DataFrame(tlist2)

In [ ]:
# Get datafram info
CoronavirusWalmartPriceMatching_df.info()

In [ ]:
# Save to excel file
CoronavirusWalmartPriceMatching_df.to_excel("CoronavirusWalmartPriceMatching.xlsx")

In [ ]:
df1 = pd.read_excel("PriceMatching.xlsx")
df2 = pd.read_excel("CoronavirusWalmartPriceMatching.xlsx")

In [ ]:
# merge all dataframes
df_list = [df1,df2]
df = pd.concat(df_list)

In [ ]:
df.head()

In [ ]:
df.info()

# CLEAN DATA

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df=df.drop(['Unnamed: 0', 'data-item-id', 'data-conversation-id','avatar'], axis=1)

In [ ]:
df.head()

In [ ]:
df['tweet'].nunique()

In [ ]:
# load library
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# download the set of stop words the first time
import nltk
nltk.download('stopwords')

In [ ]:
# Load stop words
stop_words = stopwords.words('english')

# Show stop words
stop_words[:10]

In [ ]:
import re 
from textblob import TextBlob 
import emoji  


import re 
from textblob import TextBlob 
import emoji  

def clean_tweet(text): 
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # remove @mentions
    text = re.sub(r'#', '',  str(text)) # remove the '#' symbol
    text = re.sub(r'RT[\s]+', '',  str(text)) # remove RT
    text = re.sub(r'https?\/\/S+', '',  str(text)) # remove the hyperlink
    text = re.sub(r'http\S+', '',  str(text)) # remove the hyperlink
    text = re.sub(r'www\S+', '',  str(text)) # remove the www
    text = re.sub(r'twitter+', '',  str(text)) # remove the twiiter
    text = re.sub(r'pic+', '',  str(text)) # remove the pic
    text = re.sub(r'com', '',  str(text)) # remove the pic

    return text


def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)



In [ ]:
df['cleaned_tweets']=df['tweet'].apply(clean_tweet)

In [ ]:
df['cleaned_tweets']=df['cleaned_tweets'].apply(remove_emoji)

In [ ]:
# Remove stop words
df['cleaned_tweets'] = df['cleaned_tweets'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

In [ ]:
df.head()

# WORD CLOUD

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in df.cleaned_tweets: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 1000, height = 800, 
                background_color='white', colormap='Set2', 
                collocations=False, 
                stopwords = stopwords, 
                min_font_size = 12).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (10, 10), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
df.to_csv("Walmart.csv")

# Sentiment Analysis

In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
def getSubjectivity(text):
    return TextBlob( str(text)).sentiment.subjectivity

def getPolarity(text):
    return TextBlob( str(text)).sentiment.polarity

In [ ]:
df.dropna(subset=['cleaned_tweets'], inplace = True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Subjectivity'] = df['cleaned_tweets'].apply(getSubjectivity)
df['Polarity'] = df['cleaned_tweets'].apply(getPolarity)
df.head()

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def get_Polarity_Analysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'
def get_Subjectivity_Analysis(score):
    if score >  0:
      return 'Opinion'
    else:
      return 'Fact'

df['Analysis_Polarity'] = df['Polarity'].apply(get_Polarity_Analysis)

df['Analysis_Subjectivity'] = df['Subjectivity'].apply(get_Subjectivity_Analysis)

# Show the dataframe
df.head()



In [ ]:
df.to_csv("Walmart.csv")

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=2)

In [ ]:
plt.figure(figsize=(15,10)) 

# plt.style.use('seaborn-pastel')

plt.scatter(df['Polarity'], df['Subjectivity'], c=df['Polarity'], s=100, cmap='RdYlGn') 

plt.xlim(-1.1, 1.1)
plt.ylim(-0.1, 1.1) 
plt.title('Sentiment Analysis') 
plt.xlabel('Polarity') 
plt.ylabel('Subjectivity') 
plt.show(),

In [ ]:
# Show the value counts
df['Analysis_Polarity'].value_counts()

In [ ]:
# Plotting and visualizing the counts
plt.figure(figsize=(15,10)) 

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Polarity'].value_counts().plot(kind = 'bar')
plt.show()

# Sentiment Analysis Pie Chart

In [ ]:
positive_polarity = [p for p in df['Polarity'] if p>0]
negative_polarity = [n for n in df['Polarity'] if n<0]
neutral_polarity = [r for r in df['Polarity'] if r==0]

total_size = len(positive_polarity) + len(negative_polarity) + len(neutral_polarity)
n_size = len(negative_polarity)/total_size
p_size = len(positive_polarity)/total_size
r_size = len(neutral_polarity)/total_size

labels = ['Neutral tweets', 'Positive tweets', 'Negative tweets']
sizes = [r_size, p_size, n_size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
	        shadow=False, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
# Plotting and visualizing the counts
plt.figure(figsize=(15,10)) 

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Subjectivity'].value_counts().plot(kind = 'bar')
plt.show()

# Topic Modeling